In [2]:
from crewai import Agent, Task, Crew #load the agent configuration from crewai

In [3]:
from dotenv import load_dotenv #load the .env file which i created. This file has my personal OpenAI key. File saved in same local folder as crewai on my c: drive
import os
load_dotenv()

True

In [4]:
import os

# Retrieve the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print("API Key loaded successfully!")
else:
    print("Failed to load API Key.") #confirm if my API key loaded successfully

API Key loaded successfully!


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

response = client.chat.completions.create(
    model="gpt-4o",  # use a current, lowercase model name
    messages=[{"role": "user", "content": "Hello, OpenAI!"}],
    max_tokens=5
)

print(response.choices[0].message.content.strip())  #testing the API for chatgpt 4 works correctly


Hello! How can I


In [6]:
# configure the agent, role and tasks
Recipeplanner= Agent( # first agent
    role="Recipe Planner",
    goal="Produce a ingredient list (qty + units) for specified {meal}",
    backstory="You are an experienced award winning chef "
              "and you know all about cooking the tastiest: {meal}."
              "You check for the most highly rated recipes"
              " to cook the: {meal} "
              "and make informed decisions to create an ingredients list. "
              "Your work is the basis for "
              "The Price Scout to find the best prices for the ingredients you listed.",
    allow_delegation=False,
	verbose=True
)

In [7]:
PriceScout = Agent( #second agent
    role="Find purchasable products for each ingredient",
    goal="Map each ingredient line to 3 candidate SKUs per store (Instacart, Amazon Fresh, Walmart, Target, local chains), with live or cached prices "
         "to cook each meal: {meal}",
    backstory="A street-smart bargain hunter that compares stores and knows store brand vs premium and is able to get the prices for the ingredients for each meal: {meal} "
                           "You base your ingredient mapping on the list given to you by the recipe planner."
              "you supply a list of ingredients, SKU's, stores and prices for the basket optimizer to build the most affordable basket(s)",
    
   allow_delegation=False,
    verbose=True
)

In [8]:
Basketoptimizer = Agent( #third agent
    role="Build the cheapest/best basket subject to your constraints.",
    goal="Choose SKUs to satisfy quantities, minimize total delivered cost "
         "limit store count to less than or equal to 2 stores ",
    backstory="You are a supply-chain optimizer with a soft spot for good olive oil "
              "Your goal is show which ingredients should be purchased from which store"
              "to ensure we see the value you are creating, ensure you show what the price would be if ingredients were puourchased from a more expensive store,"
              "show the total cost of the basket of chosen SKU's",
    allow_delegation=False,
    verbose=True
)

In [9]:
checkout = Agent( #forth agent
    role="make it actionable and tidy",
    goal="Summarize the plan, generate shareable ingredients list, provide links, images from the stores.",
         
    backstory="You are a crisp executive assistant who hates messy lists",
    
    allow_delegation=False,
    verbose=True
)

In [10]:
plan = Task( # task for the 1st agent
    description=(
        "1. check for highly rated recipes for the meal from top cooking websites and blogs.\n"
        
        "2. identify key reviews for the recipe to justify your choice.\n"
           
        "3. Develop a detailed list of ingredients and quantities for the meal.\n "
           
          ),
    expected_output="A comprehensive ingredient list",
       
    agent=Recipeplanner,
)

In [11]:
search = Task( #task for the second agent
    description=(
        "1. check for 3 different SKU's for each of the ingredients in the list generated by the Recipeplanner.\n"
        
        "2. icheck for these SKU's across popular online stores in the area including walmart, instacart, amazonfresh, shaws and other online grocery stores.\n"
           
        "3. Develop a detailed mapping of these SKU's with prices, stores, price per 100g, availability and URL.\n "
           
          ),
    expected_output="A comprehensive list of ingredients with prices SKU and store mapping",
       
    agent=PriceScout,
)

In [12]:
collate= Task( #task for the third agent
    description=(
        "1. build the most affordable/optimal basket of ingredients based on the list provided by the Pricescout.\n"
        
        "2. limit the number of baskets to a maximum of 2 stores only as required.\n"
           
        "3. create a comparison table to highlight cost savings gained from your selection.\n "
           
          ),
    expected_output="list of chosen SKUs per ingredient, store splits, total cost",
       
    agent=Basketoptimizer
)

In [13]:
present = Task( #task for the forth agent
    description=(
        "1. arrange the list of the chosen SKUs, the meal plan, total cost, savings and links in a visually appealing document. \n"
        
       
           
          ),
    expected_output="high level summary showing the meal, ingredients list, total cost, link to SKU's",
       
    agent=checkout
)

In [14]:
crew = Crew( #creating a crew of 4 agents
    agents=[Recipeplanner, PriceScout, Basketoptimizer, checkout],
    tasks=[plan, search, collate, present],
    verbose=True
)

In [15]:
result = crew.kickoff(inputs={"meal": "nigerian jollof rice"}) #testing to see if the crew works. the demo meal is Nigerian Jollof rice

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f88c9809-62b1-4d31-a5c2-9060af896e25                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Recipe Planner                                                                                          │
│                                                                                                                 │
│  Task: 1. check for highly rated recipes for the meal from top cooking websites and blogs.                      │
│  2. identify key reviews for the recipe to justify your choice.                                                 │
│  3. Develop a detailed list of ingredients and quantities for the meal.                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Recipe Planner                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is a comprehensive ingredient list for cooking delicious Nigerian Jollof Rice based on highly-rated       │
│  recipes from top cooking websites:                                                                             │
│                                                                                                                 │
│  ### Ingredients List for Nigerian Jollof Rice:                                                                 │
│                                                                                                                 │
│  1. **Long Grain Parboiled Rice** - 4 cups (about 800g)                                                         │
│  2. **Tomatoes (fresh)** - 6 medium-sized (or alternatively, 2 cans of crushed tomatoes, 400g each)             │
│  3. **Tomato Paste** - 1/4 cup (about 60g)                                                                      │
│  4. **Red Bell Peppers** - 3 medium-sized                                                                       │
│  5. **Onion** - 2 large (1 for the sauce, 1 for cooking)                                                        │
│  6. **Garlic** - 4 cloves, minced                                                                               │
│  7. **Ginger** - 1-inch piece, grated                                                                           │
│  8. **Scotch Bonnet Peppers** - 1-2 (adjust based on spice preference)                                          │
│  9. **Chicken or Vegetable Stock** - 5 cups (can substitute with water for vegan option)                        │
│  10. **Vegetable Oil** - 1/2 cup (about 120ml)                                                                  │
│  11. **Thyme (dried)** - 1 teaspoon                                                                             │
│  12. **Bay Leaves** - 2 leaves                                                                                  │
│  13. **Salt** - to taste (typically around 1-2 teaspoons)                                                       │
│  14. **Black Pepper** - 1/2 teaspoon                                                                            │
│  15. **Seasoning Cubes (Maggi or similar)** - 2 cubes                                                           │
│  16. **Green Peas (frozen or fresh)** - 1 cup (optional)                                                        │
│  17. **Carrots (diced, optional)** - 1 cup (optional)                                                           │
│  18. **Protein (Chicken, Beef, or Shrimp, optional)** - 500g, if desired                                        │
│                                                                                                                 │
│  ### Key Reviews Justifying Choices:                                                                            │
│  - Reviews highlighted the importance of using fresh ingredients for rich flavors, especially ripe red bell     │
│  peppers and tomatoes to create a vibrant Jollof.                                                               │
│  - Recipe followers emphasized the significance of the right balance of spices, particularly the use of thyme,  │
│  garlic, and ginger, which elicited rave reviews for depth of flavor.                                           │
│  - Many noted that adding a touch of stock enhances the

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7b1cafec-6e52-4fda-86b1-500b3dfedb4b                                                                     │
│  Agent: Recipe Planner                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Find purchasable products for each ingredient                                                           │
│                                                                                                                 │
│  Task: 1. check for 3 different SKU's for each of the ingredients in the list generated by the Recipeplanner.   │
│  2. icheck for these SKU's across popular online stores in the area including walmart, instacart, amazonfresh,  │
│  shaws and other online grocery stores.                                                                         │
│  3. Develop a detailed mapping of these SKU's with prices, stores, price per 100g, availability and URL.        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 09f4f07d-4b40-4edd-8d4b-a3f36f977900                                                                     │
│  Agent: Build the cheapest/best basket subject to your constraints.                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: make it actionable and tidy                                                                             │
│                                                                                                                 │
│  Task: 1. arrange the list of the chosen SKUs, the meal plan, total cost, savings and links in a visually       │
│  appealing document.                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: make it actionable and tidy                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### High-Level Summary: Nigerian Jollof Rice Ingredients and Pricing                                           │
│                                                                                                                 │
│  Prepare to cook an authentic and delightful dish of Nigerian Jollof Rice with the comprehensive ingredient     │
│  list below. Each entry includes price details to ensure a budget-friendly experience while maximizing flavor.  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### Ingredients List for Nigerian Jollof Rice:                                                                │
│  1. **Long Grain Parboiled Rice (800g)**                                                                        │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 078742202004                                                                                     │
│     - **Price:** $2.98                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  2. **Tomatoes (6 medium-sized, approx. 600g)**                                                                 │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 078742206428                                                                                     │
│     - **Price:** $3.99                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  3. **Tomato Paste (60g)**                                                                                      │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 050000021140                                                                                     │
│     - **Price:** $0.85                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  4. **Red Bell Peppers (3 medium-sized)**                                                                       │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 006811000040                                                                                     │
│     - **Price:** $3.97                                 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e4e5c5b7-b2ce-4988-aebd-c5fd92da5b3e                                                                     │
│  Agent: make it actionable and tidy                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f88c9809-62b1-4d31-a5c2-9060af896e25                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### High-Level Summary: Nigerian Jollof Rice Ingredients and Pricing                             │
│                                                                                                                 │
│  Prepare to cook an authentic and delightful dish of Nigerian Jollof Rice with the comprehensive ingredient     │
│  list below. Each entry includes price details to ensure a budget-friendly experience while maximizing flavor.  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### Ingredients List for Nigerian Jollof Rice:                                                                │
│  1. **Long Grain Parboiled Rice (800g)**                                                                        │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 078742202004                                                                                     │
│     - **Price:** $2.98                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  2. **Tomatoes (6 medium-sized, approx. 600g)**                                                                 │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 078742206428                                                                                     │
│     - **Price:** $3.99                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  3. **Tomato Paste (60g)**                                                                                      │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 050000021140                                                                                     │
│     - **Price:** $0.85                                                                                          │
│     - **[Walmart Link](https://www.walmart.com)**                                                               │
│                                                                                                                 │
│  4. **Red Bell Peppers (3 medium-sized)**                                                                       │
│     - **Store:** Walmart                                                                                        │
│     - **SKU:** 006811000040                           

In [16]:
import os #confirming what files are associated with this project, specifically the .env, which has my personal GPT API. This should not be exported due to security reasons

# List all .env-like files under the current folder
for root, dirs, files in os.walk("."):
    for f in files:
        if f.lower() == ".env" or f.lower().startswith(".env."):
            print(os.path.join(root, f))


.\.env
.\.jupyter\.env
.\.jupyter\lab\workspaces\.env
.\AppData\Roaming\Microsoft\Windows\Recent\.env.lnk


In [17]:
content = """# keep secrets & junk out of Git 
.env
.venv/
venv/
__pycache__/
.ipynb_checkpoints/
*.pyc
.DS_Store
results_cache/private/
"""
with open(".gitignore", "w", encoding="utf-8") as f:
    f.write(content)
print("Created .gitignore")
# created the .gitignore file so that my .env file is not exposed when i export to github

Created .gitignore


In [18]:
import os
print("Files containing 'gitignore':", [f for f in os.listdir(".") if "gitignore" in f.lower()])

with open(".gitignore", "r", encoding="utf-8") as f:
    print("\n--- .gitignore contents ---\n" + f.read())


Files containing 'gitignore': ['.gitignore']

--- .gitignore contents ---
# keep secrets & junk out of Git 
.env
.venv/
venv/
__pycache__/
.ipynb_checkpoints/
*.pyc
.DS_Store
results_cache/private/



In [19]:
import os, pathlib
print("Working folder:", os.getcwd())                 # confirmed exactly where the file was created
print("gitignore exists?", pathlib.Path(".gitignore").exists())
print("Full path:", pathlib.Path(".gitignore").resolve())


Working folder: C:\Users\adet_
gitignore exists? True
Full path: C:\Users\adet_\.gitignore


In [20]:
from pathlib import Path

p = Path(".gitignore")
print("Exists:", p.exists(), "| Is file:", p.is_file(), "| Full path:", p.resolve())
if p.exists() and p.is_file():
    try:
        print("\n--- .gitignore contents ---\n")
        print(p.read_text(encoding="utf-8"))
    except Exception as e:
        print("Couldn't read .gitignore:", e)


Exists: True | Is file: True | Full path: C:\Users\adet_\.gitignore

--- .gitignore contents ---

# keep secrets & junk out of Git 
.env
.venv/
venv/
__pycache__/
.ipynb_checkpoints/
*.pyc
.DS_Store
results_cache/private/



In [21]:
from pathlib import Path
print("--- .gitignore now contains ---\n")
print(Path(".gitignore").read_text(encoding="utf-8"))


--- .gitignore now contains ---

# keep secrets & junk out of Git 
.env
.venv/
venv/
__pycache__/
.ipynb_checkpoints/
*.pyc
.DS_Store
results_cache/private/

